In [1]:
# importing the required modules
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import nltk 

In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
#loading fake and real data 
df_fake = pd.read_csv("sample_data/Fake.csv")
df_true = pd.read_csv("sample_data/True.csv")

In [4]:
df_fake.head(5)

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [5]:
df_true.head(5)

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [6]:
# creating a new column - class with 0 as value for fake and 1 for true
df_fake["class"] = 0
df_true["class"] = 1

In [7]:
# mergig fake and true data rowwise
df_merge = pd.concat([df_fake, df_true], axis =0 )
df_merge.head(10)

,title,text,subject,date,class
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0
5,Racist Alabama Cops Brutalize Black Boy While...,The number of cases of cops brutalizing and ki...,News,"December 25, 2017",0
6,"Fresh Off The Golf Course, Trump Lashes Out A...",Donald Trump spent a good portion of his day a...,News,"December 23, 2017",0
7,Trump Said Some INSANELY Racist Stuff Inside ...,In the wake of yet another court decision that...,News,"December 23, 2017",0
8,Former CIA Director Slams Trump Over UN Bully...,Many people have raised the alarm regarding th...,News,"December 22, 2017",0
9,WATCH: Brand-New Pro-Trump Ad Features So Muc...,Just when you might have thought we d get a br...,News,"December 21, 2017",0


In [8]:
# deleting 3 columns 
df = df_merge.drop(["title", "subject","date"], axis = 1)

In [9]:
df.isnull().sum()

text     0
class    0
dtype: int64

In [10]:
# shuffling
df = df.sample(frac = 1)

In [11]:
df.head()

,text,class
17332,EDINBURGH (Reuters) - The British and Scottish...,1
550,WASHINGTON (Reuters) - President Donald Trump ...,1
12141,TORONTO (Reuters) - Canadian police said they ...,1
1965,WASHINGTON (Reuters) - Defense Secretary Jim M...,1
3508,BERLIN (Reuters) - Europe must make clear to t...,1


In [12]:
# resetting index 
df.reset_index(inplace = True)
df.drop(["index"], axis = 1, inplace = True)

In [13]:
df.columns

Index(['text', 'class'], dtype='object')

In [14]:
df.head()

,text,class
0,EDINBURGH (Reuters) - The British and Scottish...,1
1,WASHINGTON (Reuters) - President Donald Trump ...,1
2,TORONTO (Reuters) - Canadian police said they ...,1
3,WASHINGTON (Reuters) - Defense Secretary Jim M...,1
4,BERLIN (Reuters) - Europe must make clear to t...,1


In [15]:
## Tokenization --> Stemming --> Stopword removal

In [16]:
# Tokenization 
from nltk.tokenize import word_tokenize
df['text'] = df['text'].apply(word_tokenize)

In [17]:
df.head()

,text,class
0,"[EDINBURGH, (, Reuters, ), -, The, British, an...",1
1,"[WASHINGTON, (, Reuters, ), -, President, Dona...",1
2,"[TORONTO, (, Reuters, ), -, Canadian, police, ...",1
3,"[WASHINGTON, (, Reuters, ), -, Defense, Secret...",1
4,"[BERLIN, (, Reuters, ), -, Europe, must, make,...",1


In [18]:
# Stemming 
from nltk.stem.snowball import SnowballStemmer
porter = SnowballStemmer('english', ignore_stopwords=False)


In [19]:
def stem_it(text):
  return [porter.stem(word) for word in text]

In [20]:
df['text'] = df['text'].apply(stem_it)

In [21]:
df.head()

,text,class
0,"[edinburgh, (, reuter, ), -, the, british, and...",1
1,"[washington, (, reuter, ), -, presid, donald, ...",1
2,"[toronto, (, reuter, ), -, canadian, polic, sa...",1
3,"[washington, (, reuter, ), -, defens, secretar...",1
4,"[berlin, (, reuter, ), -, europ, must, make, c...",1


In [22]:
# Stopwords

def stop_it(t):
  dt = [word for word in t if len(word)>2]
  return dt 

In [23]:
df['text'] = df['text'].apply(stop_it)

In [24]:
df.head()

,text,class
0,"[edinburgh, reuter, the, british, and, scottis...",1
1,"[washington, reuter, presid, donald, trump, de...",1
2,"[toronto, reuter, canadian, polic, said, they,...",1
3,"[washington, reuter, defens, secretari, jim, m...",1
4,"[berlin, reuter, europ, must, make, clear, the...",1


In [31]:
df['text'] = df['text'].apply(' '.join)

In [32]:
# text and class into two variables
x = df["text"]
y = df["class"]

In [33]:
# splitting of data into training and testing datasets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)
x_train.head()

9447     madrid reuter some 844 peopl have been injur d...
747      the man who serv white hous counsel former pre...
37298    moscow reuter russian tu-22m3 long-rang bomber...
8699     senat minor leader harri reid has out for repu...
1132     kinshasa reuter about peopl were kill and more...
Name: text, dtype: object

In [34]:
# vectorization 
from sklearn.feature_extraction.text import TfidfVectorizer

In [35]:
vectorization = TfidfVectorizer(max_df = 0.7)
xv_train = vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_test)

In [36]:
# Linear Regression
from sklearn.linear_model import LogisticRegression

In [37]:
LR = LogisticRegression()
LR.fit(xv_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [38]:
pred_lr=LR.predict(xv_test)

In [39]:
LR.score(xv_test, y_test)

0.9869933184855234

In [40]:
print(classification_report(y_test, pred_lr))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5879
           1       0.98      0.99      0.99      5346

    accuracy                           0.99     11225
   macro avg       0.99      0.99      0.99     11225
weighted avg       0.99      0.99      0.99     11225



In [41]:
# Passive Aggressive Classifier
from sklearn.linear_model import PassiveAggressiveClassifier

model = PassiveAggressiveClassifier(max_iter=50)
model.fit(xv_train, y_train)

PassiveAggressiveClassifier(C=1.0, average=False, class_weight=None,
                            early_stopping=False, fit_intercept=True,
                            loss='hinge', max_iter=50, n_iter_no_change=5,
                            n_jobs=None, random_state=None, shuffle=True,
                            tol=0.001, validation_fraction=0.1, verbose=0,
                            warm_start=False)

In [42]:
pred_pac = model.predict(xv_test)

In [43]:
model.score(xv_test, y_test)

0.996347438752784

In [44]:
print(classification_report(y_test, pred_pac))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5879
           1       1.00      1.00      1.00      5346

    accuracy                           1.00     11225
   macro avg       1.00      1.00      1.00     11225
weighted avg       1.00      1.00      1.00     11225

